<a href="https://colab.research.google.com/github/Vannora9/Polarity_Thesis/blob/main/2021_11_27_BOW%2BSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import pandas as pd
import numpy as np
import sklearn.svm as svm
import multiprocessing

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn import preprocessing
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/program/datasets/data.csv", encoding='latin-1', error_bad_lines=False, dtype="str")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
print(data)

     lable                                           sentence
0        0  the movie begins in the past where a young boy...
1        0  emerging from the human psyche and showing cha...
2        0  spurning her mother's insistence that she get ...
3        0  amitabh can't believe the board of directors a...
4        0                                               she 
...    ...                                                ...
9995     1                                         a haunted 
9996     1      the movie's gloomy atmosphere is fascinating 
9997     1  it aimlessly and unsuccessfully attempts to fu...
9998     1                            an authentically vague 
9999     1                              visually imaginative 

[10000 rows x 2 columns]


In [ ]:
for i in data['sentence'][:300]:
  print(i)
#print(data['sentence'][:300])

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism 
spurning her mother's insistence that she get on with her life 
amitabh can't believe the board of directors and his mind is filled with revenge and what better revenge than robbing the bank himself 
she 
this gives the girls a fair chance of pulling the wool over their eyes using their sexiness to poach any last vestige of common sense the dons might have had . 
styled after vh1's " behind the music 
being blue is not his only predicament ; he also lacks the ability to outwardly express his emotions . 
the killer's clues are a perversion of biblical punishments for sins : stoning 
david is a painter with painter's block who takes a job as a waiter to get some inspiration . 
women craved him and men wanted to be him . 
set on an island off the coast of florida 
lesson to be learned : never 
the theme of t

In [ ]:
sub_lable=data["lable"]

In [ ]:
# Load stop-words
stop_words = set(stopwords.words('english'))

# Initialize tokenizer
regexp_tokenizer = RegexpTokenizer('[\'a-zA-Z]+')

# Initialize lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
def tokenize(document, rebuild_document=True):
    words = []

    for sentence in sent_tokenize(document):
        tokens = [wordnet_lemmatizer.lemmatize(t.lower()) 
        for t in regexp_tokenizer.tokenize(sentence) if t.lower() not in stop_words]
        words += tokens

    if rebuild_document:
        return ' '.join(words).strip()
    else:
        return words

In [ ]:
sub_data = []

df = pd.DataFrame(data)             #类似excel，是一种二维表
df = np.array(df)
df = df.astype(str)
df1 = df.tolist()
for i in range(0, len(data)):
    sub_data.append(tokenize(df1[i][1]))
  
for i in sub_data[:300]:
  print(i)
#print(sub_data)

movie begin past young boy named sam attempt save celebi hunter
emerging human psyche showing characteristic abstract expressionism
spurning mother's insistence get life
amitabh can't believe board director mind filled revenge better revenge robbing bank

give girl fair chance pulling wool eye using sexiness poach last vestige common sense don might
styled vh 's behind music
blue predicament also lack ability outwardly express emotion
killer's clue perversion biblical punishment sin stoning
david painter painter's block take job waiter get inspiration
woman craved men wanted
set island coast florida
lesson learned never
theme film simultaneously address similarity two faction law crime revealing similarity two brother
they're jewish
he's neglecting work
going
valento feel heat turn table make dupe one rub da's nose
saigon
deep northwest
young teenager
life
game chasing
television made famous
jordan mom life long search true faith try protect child belief injustice
'bloody magic' story 

In [ ]:
def bow(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(data)  #学习出文本的字典
  sub_data_bow = tokenizer.texts_to_sequences(data) #通过这个dict可以将每个string的每个词转成数字
  sub_data_bow= pad_sequences(sub_data_bow, maxlen=200)   #通过padding的方法补成同样长度
  return sub_data_bow

In [ ]:
X_data_bow=bow(sub_data)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_data_bow, sub_lable, test_size=0.3)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() #StandardScaler类是一个用来讲数据进行归一化和标准化的类
svc = SVC()

# Define pipeline
svc_pipeline = Pipeline([('scaler', scaler), ('svc', svc)])
# Grid search parameters
svc_param_grid = [
    #{ 
     #   'svc__kernel': [ 'linear' ],
     #   'svc__C': [0.5, 1.0, 1.3, 1.5, 1.8, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0, 10.0 ]
    #},
    #{ 
    #    'svc__kernel': [ 'poly' ],
    #    'svc__C': [0.5, 1.0, 1.3, 1.5, 1.8, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0, 10.0 ]
    #},
    { 
        'svc__kernel': [ 'rbf' ],
        'svc__C': [0.5, 1.0, 1.3, 1.5, 1.8, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0, 10.0, 13, 15]
    }
]

# Perform grid search
svc_gs = GridSearchCV(estimator=svc_pipeline, param_grid=svc_param_grid, 
                           scoring='accuracy', cv=15, n_jobs=multiprocessing.cpu_count())


svc_gs.fit(X_train, y_train)

# Create an instance of the best estimator
svc_best = svc_gs.best_estimator_
svc_best.fit(X_train, y_train)


print('Best model: %s' % str(svc_gs.best_params_))
print('Best score: %f' % svc_gs.best_score_)
print('Best test accuracy: %s' % svc_gs.score(X_val, y_val))

Best model: {'svc__C': 13, 'svc__kernel': 'rbf'}
Best score: 0.690570
Best test accuracy: 0.689


**----------------------------------------------------------------------------------**

In [ ]:
def truncatedvectors(data,n_components=300):
  svd_ngram = TruncatedSVD(n_components=n_components, random_state=42)
  svd_ngram.fit(data)
  return svd_ngram.transform(data)

In [ ]:
def tfidf(data):
  tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(1, 2))
  tfidf_vect_ngram.fit(data)
  X_data_tfidf_ngram =  tfidf_vect_ngram.transform(data)
  return truncatedvectors(X_data_tfidf_ngram)

In [ ]:
X_data_tfidf_cmt=tfidf(sub_data)
print(X_data_tfidf_cmt)

[[ 0.01559359  0.04743816  0.07348481 ...  0.01594385  0.0057213
  -0.00764764]
 [ 0.0015858   0.00495579  0.00267961 ... -0.00431148  0.00282505
   0.00511675]
 [ 0.01355419  0.03572608  0.02591722 ...  0.005001   -0.0054985
  -0.00818447]
 ...
 [ 0.02409557  0.07280912  0.04927193 ... -0.01352181 -0.00096981
   0.02926702]
 [ 0.00029066  0.00093106  0.00183889 ...  0.00170857 -0.00485002
  -0.00240502]
 [ 0.00084806  0.00269119  0.00125949 ...  0.00631811 -0.02503989
   0.0018883 ]]


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_data_tfidf_cmt, sub_lable, test_size=0.3, random_state=42)

--------------------------------------------------------------------------------------------------------------------------


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() #StandardScaler类是一个用来讲数据进行归一化和标准化的类
svc = SVC()

# Define pipeline
svc_d2v_pipeline = Pipeline([('scaler', scaler), ('svc', svc)])
# Grid search parameters
svc_d2v_param_grid = [
    { 
        'svc__kernel': [ 'linear' ],
        'svc__C': [0.5, 1.0, 1.3, 1.5, 1.8, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0, 10.0 ]
    },
    { 
        'svc__kernel': [ 'poly' ],
        'svc__C': [0.5, 1.0, 1.3, 1.5, 1.8, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0, 10.0 ]
    },
    { 
        'svc__kernel': [ 'rbf' ],
        'svc__C': [0.5, 1.0, 1.3, 1.5, 1.8, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0, 10.0 ]
    }
]

# Perform grid search
svc_d2v_gs = GridSearchCV(estimator=svc_d2v_pipeline, param_grid=svc_d2v_param_grid, 
                           scoring='accuracy', cv=15, n_jobs=multiprocessing.cpu_count())


svc_d2v_gs.fit(X_train, y_train)

# Create an instance of the best estimator
svc_d2v_best = svc_d2v_gs.best_estimator_
svc_d2v_best.fit(X_train, y_train)


print('Best model: %s' % str(svc_d2v_gs.best_params_))
print('Best score: %f' % svc_d2v_gs.best_score_)
print('Best test accuracy: %s' % svc_d2v_gs.score(X_val, y_val))